# Corrections based on sky emission lines

Compute relative offsets in wavelength (in pixels) and flux (arbitrary units) based on the sky emission lines, detected through a wavelet filter

# 1. Initialisation

In [ ]:
%matplotlib ipympl
from matplotlib import pyplot as plt
import numpy as np

In [ ]:
from pykoala import __version__
from pykoala.instruments import koala_ifu, weave
from pykoala.corrections import sky
from pykoala.corrections.throughput import ThroughputCorrection
from pykoala.corrections.wavelength import WavelengthCorrection
print("pyKOALA version: ", __version__)

The following will probably disappear in the final version of the tutorial

In [ ]:
import importlib
from astropy import stats
from astropy import units as u
from pykoala.plotting.utils import new_figure, plot_image, plot_fibres
from matplotlib.colors import LogNorm
import scipy

In [ ]:
%load_ext autoreload
%autoreload 2

# 2. Load the science data
This must be a Row-Stacked Spectra (RSS) file. Please choose one of the following examples

In [ ]:
#example = 'KOALA'
example = 'WEAVE'

KOALA:

In [ ]:
if example == 'KOALA':
    filename = f"../data/27feb20036red.fits"
    rss0 = koala_ifu.koala_rss(filename)
    wavelength_AA = rss0.wavelength.to_value(u.Angstrom)

WEAVE:

In [ ]:
if example == 'WEAVE':
    #filename = f"../data/weave/single_3042890.fit"
    filename = "../data/weave/v0.91/OB11162/single_3063947.fit"
    rss0 = weave.weave_rss(filename)
    wavelength_AA = rss0.wavelength.to_value(u.Angstrom)

## Summary

In [ ]:
print(f"Analysing object {rss0.info['name']} read from {filename}")
print('- info:')
print(rss0.info.keys())
print('- log:')
rss0.history.show()


# 3. Wavelet filter

In [ ]:
importlib.reload(sky)

## First iteration

In [ ]:
wavelet1 = sky.WaveletFilter(rss0)

In [ ]:
throughput_corr = ThroughputCorrection(throughput=wavelet1.get_throughput_object())

In [ ]:
wave_corr = WavelengthCorrection(offset=wavelet1.get_wavelength_offset())

In [ ]:
rss1 = wave_corr.apply(rss0)
rss1 = throughput_corr.apply(rss1)

## Second iteration

In [ ]:
wavelet2 = sky.WaveletFilter(rss1)

In [ ]:
throughput_corr = ThroughputCorrection(throughput=wavelet2.get_throughput_object())

In [ ]:
wave_corr = WavelengthCorrection(offset=wavelet2.get_wavelength_offset())

In [ ]:
rss2 = wave_corr.apply(rss1)
rss2 = throughput_corr.apply(rss2)

In [ ]:
rss2.history.show()

# 4. Sky subtraction

In [ ]:
importlib.reload(sky)

In [ ]:
sky0_fit = sky.SkyFromObject(rss0, bckgr_estimator='fit', bckgr_params={'wavelet': wavelet1})

In [ ]:
sky_fit = sky.SkyFromObject(rss1, bckgr_estimator='fit', bckgr_params={'wavelet': wavelet1})

In [ ]:
sky2_fit = sky.SkyFromObject(rss2, bckgr_estimator='fit', bckgr_params={'wavelet': wavelet2})

In [ ]:
sky_med = sky.SkyFromObject(rss1, bckgr_estimator='percentile')

In [ ]:
sky_mad = sky.SkyFromObject(rss1, bckgr_estimator='mad', source_mask_nsigma=3)

In [ ]:
fig, axes = new_figure('sky model')

ax = axes[0, 0]

ax.plot(wavelength_AA, sky_med.intensity, 'b-', alpha=.5, label='median')
std = np.sqrt(sky_med.variance)
ax.fill_between(wavelength_AA, sky_med.intensity-std, sky_med.intensity+std, color='b', alpha=.1, label='$16-84$ \\%')

ax.plot(wavelength_AA, sky_mad.intensity, 'b--', alpha=.5, label='MAD + 3-$\\sigma$ clip')

ax.plot(wavelength_AA, sky0_fit.intensity, 'k:', alpha=.5, label='uncorrected fit')
ax.plot(wavelength_AA, sky_fit.intensity, 'k-', alpha=.5, label='fit')
ax.plot(wavelength_AA, sky2_fit.intensity, 'r-', alpha=.5, label='iter 2')

ax.legend()
,

# 5. Quality control plots

## Wavlelet filter

In [ ]:
wavelet1.qc_plots()

## Relative calibration

In [ ]:
fig, axes = new_figure('sky-based relative calibration', nrows=2)

ax = axes[0, 0]
ax.set_ylabel('relative throughput')
#ax.set_ylim(.4, 2.1)

ax.plot(wavelet1.fibre_throughput, 'k-', label='iter 1')
ax.plot(wavelet2.fibre_throughput, 'k-', alpha=.25, label='iter 2')

#p16, p50, p84 = np.nanpercentile(wavelet1.filtered / wavelet1.sky[np.newaxis, :], [16, 50, 84], axis=1)
#ax.plot(p50, 'r--', alpha=.5)
#ax.fill_between(np.arange(p50.size), p16, p84, color='r', alpha=0.1)
#p16, p50, p84 = np.nanpercentile(wavelet2.filtered / wavelet2.sky[np.newaxis, :], [16, 50, 84], axis=1)
#ax.plot(p50, 'y--', alpha=.5)

ax.legend()


ax = axes[1, 0]
ax.set_ylabel('relative offset [pix]')

ax.plot(wavelet1.fibre_offset, 'k-', label='iter 1')
ax.plot(wavelet2.fibre_offset, 'k-', alpha=.25, label='iter 2')

ax.legend()
ax.set_xlabel('fibre')
,

In [ ]:
print(wavelet1.fibre_throughput.unit)

In [ ]:
fig, axes = new_figure('relative calibration maps',
                       nrows=2, ncols=2, sharey=False, figsize=(8, 10),
                       gridspec_kw={'left': 0.07, 'right':0.9, 'width_ratios': [1, .05], 'hspace': 0.05, 'wspace': 0.25})

plot_fibres(fig, axes[0, 0], rss0, data=wavelet1.fibre_throughput, cbax=axes[0, 1], cmap='Spectral', cblabel='relative throughput')
plot_fibres(fig, axes[1, 0], rss0, data=wavelet1.fibre_offset, cbax=axes[1, 1], cmap='Spectral', cblabel='relative offset')

axes[0, 0].sharey(axes[1, 0])

## Single fibre tests

In [ ]:
fibre = 42

In [ ]:
fig, axes = new_figure('single throughput', nrows=3, sharex=False, sharey=False, gridspec_kw={'hspace': .2})

ax = axes[0, 0]
ax.hist(wavelet1.filtered[fibre]*wavelet1.fibre_throughput[fibre] / wavelet1.sky, bins=np.linspace(0, 7.5, 101), color='k', alpha=.2)
ax.hist(wavelet1.filtered[fibre] / wavelet1.sky, bins=np.linspace(0, 7.5, 101), color='b', alpha=.5)

ax = axes[1, 0]
ax.set_ylabel('wavelet')
ax.plot(wavelet1.wavelength, wavelet1.sky, 'b:', alpha=.5, label='sky')
ax.plot(wavelet1.wavelength, wavelet1.filtered[fibre]*wavelet1.fibre_throughput[fibre], 'k--', alpha=.2, label='data')
ax.plot(wavelet1.wavelength, wavelet1.filtered[fibre], 'k-', alpha=.5, label='throughput-corrected')
ax.plot(wavelet2.wavelength, wavelet2.filtered[fibre], 'r-', alpha=.5, label='iter 2')
ax.legend()


ax = axes[2, 0]
ax.set_ylabel('intensity')
ax.sharex(axes[1, 0])

x = rss0.wavelength
if isinstance(x, u.Quantity):
    x = x.to_value(u.AA)

ax.plot(x, rss0.intensity[fibre], 'k--', alpha=.2, label=f'fibre {fibre}')
ax.plot(x, rss1.intensity[fibre], 'k-', alpha=.5, label='corrected')
ax.plot(x, rss1.intensity[fibre] - sky_fit.intensity, 'c-', alpha=.5, label='sky-subtracted')
ax.plot(x, rss2.intensity[fibre] - sky2_fit.intensity, 'c--', alpha=.5, label='iter 2')

ax.legend()
ax.set_xlabel(r'wavelength [$\AA$]')


In [ ]:
fig, axes = new_figure(f'single wavelength calibration')

ax = axes[0, 0]

mid = wavelet1.wavelength.size // 2
s = wavelet1.scale
x = np.nanmedian(wavelet1.filtered, axis=0)
x[~ np.isfinite(x)] = 0
x = scipy.signal.fftconvolve(wavelet1.filtered[fibre], x[::-1], mode='same')[mid-s:mid+s+1]
idx = np.arange(x.size)

ax.plot(idx - s, x/np.max(x), 'k-', label=f'fibre {fibre}')
ax.axvline(wavelet1.fibre_offset[fibre], c='k', ls=':', label=f'offset = {wavelet1.fibre_offset[fibre]:.2f} pix')
ax.plot(2*wavelet1.fibre_offset[fibre] - (idx - s), x/np.max(x), 'k--', alpha=.25, label='reflected cross-correlation')

ax.legend()
ax.set_ylabel('cross-correlation with sky')
ax.set_xlabel('offset [pix]')
,

## Corrected intensities

In [ ]:
fig, axes = new_figure('skyline-based correction',
                       nrows=2, ncols=3, sharex=False, sharey=False,
                       gridspec_kw={'left': 0.07, 'right':0.9, 'width_ratios': [1, 1, .05], 'hspace': 0.25, 'wspace': 0.25})

x = rss0.wavelength
if isinstance(x, u.Quantity):
    x = x.to_value(u.AA)

im, cb = plot_image(fig, axes[0, 0], 'intensity', rss0.intensity, x=x, ylabel='fibre', cbax=axes[0, 2])
im, cb = plot_image(fig, axes[0, 1], '.', rss1.intensity, x=x, norm=im.norm, cbax=False)

im, cb = plot_image(fig, axes[1, 0], 'subtracted', rss0.intensity - sky0_fit.intensity, x=x, cbax=axes[1, 2])
im, cb = plot_image(fig, axes[1, 1], '.', rss1.intensity - sky_fit.intensity, x=x, norm=im.norm, cbax=False)

for ax in [axes[0, 1], axes[1, 0], axes[1, 1]]:
    ax.sharex(axes[0, 0])
    ax.sharey(axes[0, 0])


## Sky subtraction (from mode)

In [ ]:
def symmetric_background(x, weights=None, fig_name=None):
    if weights is None:
        weights = np.ones_like(x)

    valid = np.where(np.isfinite(x) & np.isfinite(weights))
    n_valid = valid[0].size
    if not n_valid > 0:
        return np.nan, np.nan

    sorted_by_x = np.argsort(x[valid].flatten())
    sorted_x = x[valid].flatten()[sorted_by_x]
    sorted_weight = weights[valid].flatten()[sorted_by_x]

    cumulative_mass = np.cumsum(sorted_weight)
    total_mass = cumulative_mass[-1]
    sorted_weight /= total_mass
    cumulative_mass /= total_mass
    
    
    nbins = int(np.sqrt(4*n_valid + 1) - 1)
    m_left = np.linspace(0, 0.5, nbins)[1:-1]
    m_mid = 2 * m_left
    m_right = 0.5 + m_left

    x_left = np.interp(m_left, cumulative_mass, sorted_x)
    x_mid = np.interp(m_mid, cumulative_mass, sorted_x)
    x_right = np.interp(m_right, cumulative_mass, sorted_x)

    h = np.fmin(x_right-x_mid, x_mid-x_left)
    valid = np.where(h > 0)
    if not valid[0].size > 0:
        return np.nan, np.nan
    x_mid = x_mid[valid]
    h = h[valid]
    rho = (np.interp(x_mid+h, sorted_x, cumulative_mass) - np.interp(x_mid-h, sorted_x, cumulative_mass)) /2/h

    rho_threshold = np.nanpercentile(rho, 100*(1 - 1/np.sqrt(nbins)))
    peak_region = x_mid[rho > rho_threshold]
    index_min = np.searchsorted(sorted_x, np.min(peak_region))
    index_max = np.searchsorted(sorted_x, np.max(peak_region))
    index_mode = (index_min+index_max) // 2
    mode = sorted_x[index_mode]
    m_mode = cumulative_mass[index_mode]

    rho_bg = np.fmin(rho, np.interp(x_mid, (2*mode - x_mid)[::-1], rho[::-1], left=0, right=0))
    if m_mode <= 0.5:
        total_bg = 2 * m_mode
        threshold = sorted_x[np.clip(int(n_valid * total_bg), 0, n_valid-1)]
        contamination = np.interp(2*mode-threshold, sorted_x, cumulative_mass)
    else:
        total_bg = 2 * (1 - m_mode)
        threshold = sorted_x[np.clip(int(n_valid * (1-total_bg)), 0, n_valid-1)]
        contamination = 1 - np.interp(2*mode-threshold, sorted_x, cumulative_mass)

    if fig_name is not None:
        plt.close(fig_name)
        fig = plt.figure(fig_name, figsize=(8, 5))
        axes = fig.subplots(nrows=1, ncols=1, squeeze=False,
                            sharex='col', sharey='row',
                            gridspec_kw={'hspace': 0, 'wspace': 0},
                           )

        ax = axes[0, 0]
        ax.set_ylabel('probability density')
        #ax.set_yscale('log')
        ax.plot(x_mid, rho-rho_bg, 'b-', alpha=.5, label='p(signal)')
        ax.plot(x_mid, rho_bg, 'r-', alpha=.5, label='p(background)')
        ax.plot(x_mid, rho, 'k-', alpha=.5, label='total')


        ax.set_xlabel('value')
        L = 5 * np.abs(threshold - mode)
        vmin = np.max([mode - L, x_mid[0]])
        vmax = np.min([mode + L, x_mid[-1]])
        ax.set_xlim(vmin, vmax)

        for ax in axes.flatten():
            ax.tick_params(which='both', bottom=True, top=True, left=True, right=True)
            ax.tick_params(which='major', direction='inout', length=8, grid_alpha=.3)
            ax.tick_params(which='minor', direction='in', length=2, grid_alpha=.1)
            ax.grid(True, which='both')
            ax.axvspan(sorted_x[index_min], sorted_x[index_max], color='k', alpha=.1)
            ax.axvline(mode, c='k', ls=':', label=f'background = {mode:.4g}')
            ax.axvline(threshold, c='b', ls='-.', label=f'signal threshold = {threshold:.4g}')
            ax.axvline(2*mode - threshold, c='r', ls='-.', alpha=.5, label=f'contamination = {100*contamination/(1-total_bg):.2f}%')
            ax.legend()

        fig.suptitle(fig_name)
        fig.set_tight_layout(True)
        plt.show()
        
    return mode, threshold

In [ ]:
mode_sky = np.empty_like(rss1.intensity[0])
for idx in range(rss1.n_wavelength):
    #print(idx, rss1.wavelength[idx])
    mode_sky[idx], dummy = symmetric_background(rss1.intensity[:, idx])

## Sky subtraction

In [ ]:
flux = np.nanmean(rss1.intensity, axis=1)
mean_flux = np.nanmean(flux)
flux_cut_low = np.nanmedian(flux[flux < mean_flux])
flux_cut_hi = 2*mean_flux - flux_cut_low
flux_low = np.nanmean(flux[flux < flux_cut_low])
flux_med = np.nanmean(flux[(flux > flux_cut_low) & (flux < mean_flux)])
flux_hi = np.nanmean(flux[(flux > mean_flux) & (flux < flux_cut_hi)])

I_low = np.nanmean(rss1.intensity[flux < flux_cut_low, :], axis=0)
I_med = np.nanmean(rss1.intensity[(flux > flux_cut_low) & (flux < mean_flux), :], axis=0)
I_hi = np.nanmean(rss1.intensity[(flux > mean_flux) & (flux < flux_cut_hi), :], axis=0)
m = (I_hi - I_low) / (flux_hi - flux_low)
b = I_low - m * flux_low

sky_flux_candidate = np.arange(0, flux_cut_hi.value, .01*np.min(flux.value)) << flux.unit
sky_filtered = m[np.newaxis, :] * sky_flux_candidate[:, np.newaxis] + b[np.newaxis, :]
x = np.nancumsum(sky_filtered, axis=1)
s = wavelet1.scale
sky_filtered = (x[:, 2*s:-s] - x[:, s:-2*s]) / s
sky_filtered -= (x[:, 3*s:] - x[:, :-3*s]) / (3*s)
chi2_sky = np.nanstd(sky_filtered*wavelet1.sky_weight - wavelet1.sky, axis=1)
chi2_no_sky = np.nanstd(sky_filtered*(1 - wavelet1.sky_weight), axis=1)

sky_flux = sky_flux_candidate[np.nanargmin(chi2_no_sky)]
#sky_flux = 280 * u.adu
#sky_flux = np.mean(np.nanmedian(rss1.intensity, axis=0))
#sky_flux = background * u.adu
sky_intensity = b + m*sky_flux

In [ ]:
background, threshold = symmetric_background(flux.value, fig_name='mean intensity')

In [ ]:
fig, axes = new_figure('intensity hist')

ax = axes[0, 0]

x = np.linspace(0, 3*sky_flux, 101)

ax.hist(flux, bins=x, alpha=.5)
ax.axvline(background, c='k', ls='--', label=f'Mode = {background:.2f}')
ax.axvline(threshold, c='k', ls=':', label=f'th = {threshold:.2f}')

ax.legend()
ax.set_ylabel('number of fibres')
ax.set_xlabel(f'mean fibre intensity')
#ax.set_xscale('log')
ax.set_yscale('log')

In [ ]:
fig, axes = new_figure('get sky flux', nrows=1)

ax = axes[0, 0]
ax.plot(sky_flux_candidate, chi2_sky, 'b-')
ax.plot(sky_flux_candidate, chi2_no_sky, 'r-')
ax.axvline(sky_flux, c='k', ls=':', label=f'sky flux = {sky_flux:.2f}')
ax.legend()
,

In [ ]:
fig, axes = new_figure('fit coefficients', nrows=4)

ax = axes[0, 0]
ax.set_ylabel('m')
ax.plot(wavelength_AA, m, 'k-')
#ax.plot(wavelet1.wavelength, wavelet1.sky_weight, 'b-')
#ax.plot(rss1.wavelength, sky_intensity / np.nanmean(sky_intensity), 'c-')


ax = axes[1, 0]
ax.set_ylabel('b')
ax.plot(wavelength_AA, sky_fit.intensity, 'r-')
ax.plot(wavelength_AA, sky_intensity, 'c-')
ax.plot(wavelength_AA, b, 'k-')
ax.plot(wavelength_AA, mode_sky, 'k--')
ax.plot(wavelength_AA, sky_mad.intensity, 'r:')
#ax.set_ylim(-10, 500)


ax = axes[2, 0]
s = wavelet1.scale
ax.set_ylabel(f'wavelet b ({s} pix)')

x = np.nancumsum(sky_intensity)
sky_filtered = (x[2*s:-s] - x[s:-2*s]) / s
sky_filtered -= (x[3*s:] - x[:-3*s]) / (3*s)
ax.plot(wavelet1.wavelength, sky_filtered, 'r-', alpha=.5)
ax.plot(wavelet1.wavelength, sky_filtered * wavelet1.sky_weight, 'k-', alpha=.5)
ax.plot(wavelet1.wavelength, wavelet1.sky, 'c-', alpha=.5)

ax = axes[3, 0]
ax.plot(wavelet1.wavelength, wavelet1.sky_weight)

## Single wavelength

In [ ]:
wl = 6581
#wl = 6620
#wl = 6700
idx = np.searchsorted(wavelength_AA, wl)

In [ ]:
bg, th = symmetric_background(rss1.intensity[:, idx].value, fig_name='monochrome mode')

In [ ]:
fig, axes = new_figure('monochrome hist')

ax = axes[0, 0]

x = np.linspace(b[idx], 3*sky_intensity[idx] - 2*b[idx], 101)

ax.hist(rss1.intensity[:, idx], bins=x, alpha=.5)
ax.axvline(sky_intensity[idx], c='k', ls='--', label=f'Sky level = {sky_intensity[idx]:.2f}')
ax.axvline(bg, c='k', ls=':', label=f'mode = {bg:.2f}')

ax.legend()
ax.set_ylabel('number of fibres')
ax.set_xlabel(f'intensity at $\\lambda={wavelength_AA[idx]:.2f}$ \\AA')
#ax.set_xscale('log')
ax.set_yscale('log')

In [ ]:
fig, axes = new_figure('linear fit')

ax = axes[0, 0]

ax.plot(flux, rss1.intensity[:, idx], 'k.')
ax.axvline(mean_flux, c='r', ls='--', label=f'mean flux: {mean_flux:.2f}')
ax.axvline(flux_cut_low, c='r', ls=':', label=f'flux low: {flux_cut_low:.2f}')
ax.axvline(flux_cut_hi, c='r', ls=':', label=f'flux high: {flux_cut_hi:.2f}')

ax.axhline(bg, c='k', ls=':', label=f'mode: {bg:.2f}')
ax.axhline(th, c='k', ls='--', label=f'th: {th:.2f}')
ax.axvline(background, c='k', ls=':', label=f'mode fl: {background:.2f}')
ax.axvline(threshold, c='k', ls='--', label=f'th fl: {threshold:.2f}')

x = np.nanpercentile(flux, np.linspace(0, 100, 101))
ax.plot(x, m[idx]*x + b[idx], 'b:', label=f'm={m[idx]:.2f} b={b[idx]:.2f}')
ax.plot(u.Quantity([flux_low, flux_med, flux_hi]), u.Quantity([I_low[idx], I_med[idx], I_hi[idx]]), 'ro-')

ax.plot(sky_flux, sky_intensity[idx], 'co', label=f'Sky level = {sky_intensity[idx]:.2f}')

ax.legend()
ax.set_xlabel('mean fibre intensity')
ax.set_ylabel(f'intensity at $\\lambda={wavelength_AA[idx]:.2f}\\ \\AA$')
ax.set_xscale('log')
ax.set_yscale('log')

In [ ]:
from pykoala.plotting import utils
from matplotlib import colors
importlib.reload(utils)
fig, axes = new_figure('correction maps',
                       nrows=4, ncols=2, sharey=False, figsize=(8, 12),
                       gridspec_kw={'left': 0.07, 'right':0.9, 'width_ratios': [1, .05], 'hspace': 0.05, 'wspace': 0.25})

#(fig, ax, rss=None, x=None, y=None,
#                fibre_diam=None, data=None, 
#                patch_args={}, use_wcs=False, fix_limits=True,
#                cmap=DEFAULT_CMAP, norm=None, cbax=None, cblabel=None, 
#                norm_interval=visualization.MinMaxInterval, interval_args={},
#                stretch=visualization.LinearStretch, stretch_args={})
norm = LogNorm(vmin=1, vmax=np.nanpercentile(rss1.intensity[:, idx].value, 90))
utils.plot_fibres(fig, axes[0, 0], rss0, data=rss0.intensity[:, idx], cbax=axes[0, 1], cblabel=f'original $I_\\lambda({wavelength_AA[idx]:.2f} \\AA)$', norm=norm)
utils.plot_fibres(fig, axes[1, 0], rss1, data=rss1.intensity[:, idx], cbax=axes[1, 1], cblabel='throughput corrected', norm=norm)
utils.plot_fibres(fig, axes[2, 0], rss1, data=rss1.intensity[:, idx] - sky_fit.intensity[idx], cbax=axes[2, 1], cblabel='sky subtracted', norm=norm)
utils.plot_fibres(fig, axes[3, 0], rss1, data=rss1.intensity[:, idx] - mode_sky[idx], cbax=axes[3, 1], cblabel='mode sky', norm=norm)

for ax in axes[1:, 0]:
    ax.sharey(axes[0, 0])
